Předělávka kódů
- využití vlastních funkcí

# Import - moduly

In [2]:
import requests
from datetime import datetime
import pandas as pd
import polyline
import geopandas as gpd
from shapely.geometry import LineString
import ast
import json

# Pomocné funkce

In [3]:
def duration(data):
    """
    Vrátí dobu první iterace cesty
    """
    return data['data']['plan']['itineraries'][0]['duration']

def leg_num(data):
    """
    Vrátí počet kroků první iterace cesty
    """
    return len(data['data']['plan']['itineraries'][0]['legs'])    

def start_walk_time(iteration):
    """
    Vrátí dobu chůze z výchozího bodu na první stanici
    """
    return iteration['legs'][0]['duration']

def end_walk_time(iteration):
    """
    Vrátí dobu chůze z poslední stanice do cíle
    """
    return iteration['legs'][-1]['duration']


# Vlastní funkce

In [4]:
# --------------------------------------------- Vytvoření slovníků -----------------------------------------------------------------------------------

def build_route_dict (category, row, name_start:str, name_end:str, date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=False, Prague=False):
    """
    Vrátí slovník ve struktuře koncové tabulky (pro obce (default) a pro parkoviště)

    municipalities=True -> vzhled slovníku pro obce
    Prague=True -> slovník pro Prahu   
    
    category - kategorie - podle rozhodnutí, jestli je to nejrychlejší nebo nejméně přestupů
    row - řádek z tabulky, který vznikl z .iterrow()
    name_start - jméno sloupce "výchozí pozice"
    end_start - jméno sloupce "koncové pozice
    date - zvolené datum
    time - zvolený čas
    iteration - je část odpovědi API, která odpovídá jedné iteraci trasy
    iteration_index - index iterace (protože používám j, iteration in enumerate())
    all_coords - GPS body pro celou trasu
    transport_vehicles - seznam jedinečných dopravních prostředků
    itinerary - celý itinerář (slovy)
    geometry_legs - geometrie jednotlivých kroků pro případnou vizualizaci; dá se to celkem jednoduše přes split rozdělit na jednotlivé kroky (oddělovač ;) a prostřed + linestring (oddělovač+)
    
    """
    dictionary = {"KATEGORIE_ITERACE": category}        # nejrychlejsi/nejmene prestupu/rail, pokud nejméně přestupů = nejrychlejší, je jenom nejrychlejší
    if municipalities:
        dictionary["OKRES_KOD"] = row['OKRES_KOD']
    if Prague:
        dictionary["data_address"] = row["data_address"]
        if len(transport_vehicles) > 0:
            dictionary["ZASTAVKA_JMENO"] = iteration['legs'][1]['from']['name']
            dictionary["ZASTAVKA_lat"] = iteration['legs'][1]['from']['lat']
            dictionary["ZASTAVKA_lon"] = iteration['legs'][1]['from']['lon']
        else:
            dictionary["ZASTAVKA_lat"] = None
            dictionary["ZASTAVKA_lon"] = None
    dictionary[name_start] = row[name_start]
    dictionary[name_end] = row[name_end]
    dictionary["DATUM A CAS"] = f'{date}T{time}'           # zadaný datum a čas
    dictionary["SPICKA"] = spicka
    dictionary["DOBA"] = round(iteration['duration']/60) # minuty
    dictionary["CAS_ZACATEK"] = datetime.fromisoformat(iteration['start']).time().isoformat() # kontrolní
    dictionary["CAS_KONEC"] = datetime.fromisoformat(iteration['end']).time().isoformat()
    dictionary["DELKA"] = round(sum(leg['distance'] for leg in iteration['legs'])/1000, 1) # kilometry
    dictionary["POCET_PRESTUPU"] = transfers
    dictionary["DOPRAVNI_PROSTREDKY"] = ', '.join(s for s in transport_vehicles)
    dictionary["TRASA"] = all_coords
    # dictionary["POCET_USEKU"] = len(iteration['legs'])        # v principu duplicita, protože pocet_useku = transfers * 2 + 1 a úseky dávají víc smysl
    dictionary["CISLO_ITERACE"] = iteration_index + 1                 # jen pro kontrolu
    dictionary["KROKY"] = itinerary                              # kontrolní               
    dictionary["LINESTRINGY_ETAPY"] = geometry_legs              # geometrie jednotlivých kroků pro případnou vizualizaci
                                                        # dá se to celkem jednoduše přes split rozdělit na jednotlivé kroky (oddělovač ;) a prostřed + linestring (oddělovač+)
    return dictionary


# ----------------------------------- Rozhodnutí směru -----------------------------------------------------------------------------
def direction_decide(time:str, lat_1:float, lon_1:float, lat_2:float, lon_2:float):
    """
    rozhodne směr cesty podle denní doby, první dvojice je lokace startu pro ráno
    """
# Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
    if datetime.strptime(time, "%H:%M").hour < 12:
        from_lat, from_lon = lat_1, lon_1
        to_lat, to_lon = lat_2, lon_2
        spicka = "ranni"
    else:
        from_lat, from_lon = lat_2, lon_2
        to_lat, to_lon = lat_1, lon_1
        spicka = "odpoledni"
    return from_lat, from_lon, to_lat, to_lon, spicka

# --------------------------------------------- Funkce - přípravy na LineString -------------------------------------------------------------
# -------------------- celková trasa
def build_coordination_string(iteration):
    """
    Převedení geometrie, kterou vrací API, na sérii GPS bodů a vytvoření seznamu pro převedení do LineString geometrie

    Přizpůsobeno pro OTP, all_coords = předpřipravený seznam, do kterého se bude ukládat
    """
    all_coords = []
    for leg in iteration['legs']:
        coords = polyline.decode(leg["legGeometry"]["points"]) # přepsání hrůzostrašných písmen na lat a lon dvojice
        coords = [(lon, lat) for lat, lon in coords] # příprava na vytvoření linestring - protože linestring je lon a lat, tak jen přehození pořadí pro každou dvojici
        if all_coords:
            if coords[0] == all_coords[-1]:
                all_coords.extend(coords[:-1])
            else:
                all_coords.extend(coords)
        else:
            all_coords.extend(coords)
    return all_coords

# ------------------- string jednolivých kroků --------------------------------------------------------------------------------
def build_geometry_legs_string(iteration):
    """
    Vytvoří řetězec ve tvaru "způsob dopravy+série GPS bodů"

    e to připravené na případnou možnost obarvení trasy podle dopravního prostředku
    dá se to celkem jednoduše přes split rozdělit na jednotlivé kroky (oddělovač ,) a prostřed + linestring (oddělovač+) 
    """
    parts = []
    for leg in iteration['legs']:
        coords = polyline.decode(leg["legGeometry"]["points"]) # přepsání hrůzostrašných písmen na lat a lon dvojice
        coords = [(lon, lat) for lat, lon in coords] # příprava na vytvoření linestring - protože linestring je lon a lat, tak jen přehození pořadí pro každou dvojici
        parts.append(f'{leg["mode"]}+{(coords)}')
    return ",".join(parts)

# ---------------------------------------------------- Řetězec itineráře -------------------------------------------------------
def build_itinerary(iteration):
    """
    Vytvoří řetězec celkového itineráře cesty

    Jednolivé kroky mají formát: "způsob dopravy, trvání v min, délka úseku v m ((linka), výchozí stanice - koncová stanice)
    """
    parts = []
    for leg in iteration["legs"]:
        if leg['mode'] == 'WALK': # WALK nemá linku, takže by to házelo chyby 
            part = f"{leg['mode']}, {round(leg['duration']/60)} min, {leg['distance']} m ({leg['from']['name']} - {leg['to']['name']})"
        else:
            part = f"{leg['mode']}, {round(leg['duration']/60)} min, {leg['distance']} m  ({leg['route']['shortName']}, {leg['from']['name']} - {leg['to']['name']})"
        parts.append(part)
    return " + ".join(parts)

# ---------------------------------------------------- Dopravní prostředky ------------------------------------------------------
def build_transport_list(iteration):
    """
    Vrátí seznam jedinečných dopravních prostředků používaných v dané cestě (seřazené podle abecedy) a celkového itineráře (pro vynechání stejných tras, které mají jen jiné čekací doby)
    """
    transports = []
    transport_vehicles = []

    for leg in iteration['legs']:
        if leg['mode'] == 'WALK':
            transports.append(leg['mode'])
        else:
            transports.append(leg['route']['shortName'])
            if leg['mode'] not in transport_vehicles:
                transport_vehicles.append(leg['mode'])
    return transports, sorted(transport_vehicles)




def transfer_num(iteration):
    """
    Vrátí počet přestupů (počet segmentů veřejné dopravy mínus 1)
    """
    PT_num = sum([1 for leg in iteration['legs'] if leg['mode'] != "WALK" ])
    return max(0, PT_num - 1)

## Funkce - export do GeoJsonu

In [6]:
def CSV_GeoJson_export(adress:str, separator:str, GPSColumnName:str, OutputAdress:str):
    """
    Převede seznam GPS na LineString a soubor uloží do GeoJson
    """

    routes_df = pd.read_csv(adress, sep=separator)
    routes_df[GPSColumnName] = routes_df[GPSColumnName].apply(lambda x: LineString(ast.literal_eval(x)))
    routes_gdf = gpd.GeoDataFrame(routes_df, geometry=GPSColumnName)
    routes_gdf.to_file(OutputAdress)
    return 


# Společné části kódu

In [7]:
url = "http://localhost:8080/otp/gtfs/v1"
headers = {"Content-Type": "application/json"}

# GraphQL query
query = '''query Parking (
        $from_lat: Float!,              # definování proměnných, které se vkládají ve "variables"
        $from_lon: Float!, 
        $to_lat: Float!, 
        $to_lon: Float!,
        $Date: String!,
        $Time: String!,
        $window: Long!,
        $NumInt: Int!
        ) 
    {
    plan(
        from: {lat: $from_lat, lon: $from_lon}
        to: {lat: $to_lat, lon: $to_lon}
        date: $Date
        time: $Time                    
        searchWindow: $window           # v jakém časovém okně od zadaného času, bude hledat trasy (v s)
        numItineraries: $NumInt         # počet iterací
        transportModes: [               # definice dopravních prostředků
            {mode: WALK},              
            {mode: BUS},               
            {mode: TRAM}, 
            {mode: RAIL}, 
            {mode: SUBWAY}, 
            {mode: TROLLEYBUS}, 
            {mode: FERRY}
        ]
        ) {
            itineraries {
            start                       # datetime, timezone - Prague
            end                         # datetime, timezone - Prague 
            duration                    # in seconds
            legs {
                mode
                route {
                shortName
                }
                distance
                duration
                from {
                name
                }
                to {
                name
                }
                legGeometry { 
                length
                points
                }
            }
            }
        }
        }
'''

Dates = ["2025-11-03", "2025-11-04", "2025-11-05", "2025-11-06", "2025-11-07"]
# Dates = ["2025-11-10", "2025-11-11", "2025-11-12", "2025-11-13", "2025-11-14"]
Times = ["06:30", "07:30", "08:30", "17:00"]

# Kanceláře - parkoviště

In [ ]:
PR_Kancelare = pd.read_csv("DATA/API_zdroje/kancelare_parkoviste.csv")

routes = []
limit_time = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)

for date in Dates:
    for time in Times:
        for _, row in PR_Kancelare.iterrows():

            # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
            from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["PARKOVISTE_Y"], row["PARKOVISTE_X"], row["KANCELAR_X"], row["KANCELAR_Y"])

            payload = {
                "query": query,
                "variables": {
                    "from_lat": float(from_lat),
                    "from_lon": float(from_lon),
                    "to_lat": float(to_lat),
                    "to_lon": float(to_lon),
                    "Date": date,
                    "Time": time,
                    "window": 1800,
                    "NumInt": 5
                    },
                "operationName": "Parking"
                }
            
            # volání API
            response = requests.post(url, headers=headers, json=payload, timeout=30)
            data = response.json()
            

            # ------------------------TRANSFORMACE-------------------------

            # pomocné proměnné
            
            duration_fast = duration(data)
            leg_num_low = leg_num(data)

            fast_route = {}
            leg_low_route = {}

            for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
                all_coords = build_coordination_string(iteration)
                geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

                itinerary = build_itinerary(iteration)

                transport_lines, transport_vehicles = build_transport_list(iteration)
                # transports - konkrétní linky
                # transport_vehicles - typy dopravních prostředků
                transfers = transfer_num(iteration)
                
                # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
                time_walk_start = start_walk_time(iteration)
                time_walk_end = end_walk_time(iteration)

                # ------------ FAST ----------
                # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
                if int(iteration['duration']) < duration_fast or iteration_index == 0:
                    duration_fast = int(iteration['duration'])
                    fast_transports = transport_lines
                    fast_route = build_route_dict('nejrychlejsi', row, "PARKOVISTE", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs)

                # -------------- LOW LEG NUMBER --------------------
                # tady je to jedno, ale v případě obcí je třeba ošetřit cestu na vlak (ta může být delší a pak se to dá oseknout)
                if len(iteration['legs']) < leg_num_low and transfers > 0:
                    # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze
                    if time_walk_start <= limit_time and time_walk_end <= limit_time: 
                        # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                        leg_num_low = len(iteration['legs'])
                        leg_low_route = build_route_dict ("nejmene prestupu", row, "PARKOVISTE", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs)
            
            if leg_low_route:
                if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                    routes.append(fast_route)
                else:
                    routes.append(fast_route)
                    routes.append(leg_low_route)
            else:
                routes.append(fast_route)

routes = pd.DataFrame(routes)
routes.to_csv("DATA_HD/trasy_PR_kancelare_test.csv", sep=';', na_rep='', index= False)


## Export do GeoJsonu

In [21]:
adress = "DATA_HD/trasy_PR_kancelare_2.csv"
OutputAdress = "DATA_HD/trasy_PR_kancelare_2.geojson"

CSV_GeoJson_export(adress, ";", "TRASA", OutputAdress)

FileNotFoundError: [Errno 2] No such file or directory: 'DATA_HD/trasy_PR_kancelare_2.csv'

# Obce - kanceláře

In [ ]:
obce = pd.read_csv("DATA/API_zdroje/Obce_kancelare_oprava.csv")

# date = Dates[4]
"""
Dates = ["2025-11-03", "2025-11-04", "2025-11-05", "2025-11-06", "2025-11-07"]
Times = ["06:30", "07:30", "08:30", "17:00"] 
"""
# time = Times[0]
routes = []
rail_routes = []

errors = []

limit_time_Prague = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)
limit_time_not_Prague = 20*60 # 20 min, omezení mimo Prahu

for date in Dates:
    for time in Times[:3]:
        for _, row in obce.iterrows():
            # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
            from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["souradnice_x"], row["souradnice_y"], row["X_KANCELAR"], row["Y_KANCELAR"])

            try:    
                payload = {
                    "query": query,
                    "variables": {
                        "from_lat": float(from_lat),
                        "from_lon": float(from_lon),
                        "to_lat": float(to_lat),
                        "to_lon": float(to_lon),
                        "Date": date,
                        "Time": time,
                        "window": 3600,
                        "NumInt": 5
                        },
                    "operationName": "Parking"
                    }
                        
                # volání API
                response = requests.post(url, headers=headers, json=payload)
                data = response.json()

                # ------------------------TRANSFORMACE-------------------------
                print(f'{row["OBEC_NAZEV"]} - {row["KANCELAR"]}')

                # pomocné proměnné 

                duration_fast = duration(data)
                rail_route_fast = None 
                leg_num_low = leg_num(data)

                fast_route = {}
                leg_low_route = {}
                rail_route = {}

                for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
                    all_coords = build_coordination_string(iteration)
                    geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

                    itinerary = build_itinerary(iteration)

                    transport_lines, transport_vehicles = build_transport_list(iteration)
                    # transports - konkrétní linky
                    # transport_vehicles - typy dopravních prostředků
                    transfers = transfer_num(iteration)
                            
                    # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
                    time_walk_start = start_walk_time(iteration)
                    time_walk_end = end_walk_time(iteration)

                    # ------------ FAST ------------------------------
                    # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
                    if int(iteration['duration']) < duration_fast or iteration_index == 0:
                        duration_fast = int(iteration['duration'])
                        # fast_transports = transport_lines
                        fast_route = build_route_dict('nejrychlejsi', row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=True)
                    
                    # -------------- LOW LEG NUMBER --------------------
                    if len(iteration['legs']) < leg_num_low and transfers > 0:
                        # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze

                        # protože rozlišuji chůzi v Praze a v obci, je třeba nejdřív rozdělit směr
                        if spicka == "ranni": # trasa obce -> kancelare
                            # takže time_walk_start - má limit_time_not_Prague
                            if time_walk_start <= limit_time_not_Prague and time_walk_end <= limit_time_Prague: 
                                # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                                leg_num_low = len(iteration['legs'])
                                leg_low_route = build_route_dict("nejmene prestupu", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=True)
                        else:
                            # tady je to obrácene, tedy time_walk_start má limit_time_Prague
                            if time_walk_start <= limit_time_Prague and time_walk_end <= limit_time_not_Prague: 
                                # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                                leg_num_low = len(iteration['legs'])
                                leg_low_route = build_route_dict("nejmene prestupu", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,municipalities=True)
                        
                    # ------------- RAIL ------------------------------------
                    if "RAIL" in transport_vehicles:
                        if rail_route_fast == None or iteration['duration'] < rail_route_fast:
                            rail_route = build_route_dict("vlak", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,municipalities=True)
                            rail_route_fast = iteration['duration']

                if leg_low_route:
                    if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                        routes.append(fast_route)
                    else:
                        routes.append(fast_route)
                        routes.append(leg_low_route)
                else:
                        routes.append(fast_route)
                if rail_route:
                    rail_routes.append(rail_route)

            except Exception:
                error_dict = {
                    "OBEC_NAZEV": row["OBEC_NAZEV"],
                    "OKRES_KOD": row["OKRES_KOD"],
                    "KANCELAR": row["KANCELAR"],
                    "DATUM": date,
                    "CAS": time,
                    "from_lat": from_lat,
                    "from_lon":from_lon,
                    "to_lat": to_lat,
                    "to_lon": to_lon,
                    "data": data
                }
                errors.append(error_dict)

routes = pd.DataFrame(routes)
# adress = f"DATA_HD/trasy_obce_{date}T{time[:2]}"
adress = f"DATA_HD/trasy_obce_{date}"
routes.to_csv(f"{adress}.csv", sep=';', na_rep='', index= False)

rail_routes = pd.DataFrame(rail_routes)
rail_routes.to_csv(f"{adress}_rail.csv", sep=';', na_rep='', index= False)

if errors:
    with open (f'{adress}_errors.json', "w", encoding="UTF-8") as output_file:
        json.dump(errors, output_file, ensure_ascii=False, indent=4)
    errors = pd.DataFrame(errors)
    errors.to_csv(f'{adress}_errors.csv', sep=';', na_rep='', index= False)

Černošice - Brumlovka
Černošice - Karlín
Černošice - Smíchov
Škvorec - Brumlovka
Škvorec - Karlín
Škvorec - Smíchov
Bakov nad Jizerou - Brumlovka
Bakov nad Jizerou - Karlín
Bakov nad Jizerou - Smíchov
Benešov - Brumlovka
Benešov - Karlín
Benešov - Smíchov
Benátky nad Jizerou - Brumlovka
Benátky nad Jizerou - Karlín
Benátky nad Jizerou - Smíchov
Beroun - Brumlovka
Beroun - Karlín
Beroun - Smíchov
Brandýs nad Labem-Stará Boleslav - Brumlovka
Brandýs nad Labem-Stará Boleslav - Karlín
Brandýs nad Labem-Stará Boleslav - Smíchov
Buštěhrad - Brumlovka
Buštěhrad - Karlín
Buštěhrad - Smíchov
Bystřice - Brumlovka
Bystřice - Karlín
Bystřice - Smíchov
Bělá pod Bezdězem - Brumlovka
Bělá pod Bezdězem - Karlín
Bělá pod Bezdězem - Smíchov
Březnice - Brumlovka
Březnice - Karlín
Březnice - Smíchov
Dobrovice - Brumlovka
Dobrovice - Karlín
Dobrovice - Smíchov
Dobřichovice - Brumlovka
Dobřichovice - Karlín
Dobřichovice - Smíchov
Dobříš - Brumlovka
Dobříš - Karlín
Dobříš - Smíchov
Dolní Bousov - Brumlovka
D

In [ ]:
# CSV_GeoJson_export(f'{adress}.csv', ';', "TRASA", f'{adress}.geojson')

In [ ]:
import json
with open("DATA_HD/trasy_obce_2025-11-03T17_errors.json", encoding="UTF-8") as file:
    data2 = json.load(file)

# Praha - kanceláře

In [1]:
# GraphQL query - tady je navíc ještě GPS lokace nástupních zastávek. Když by to ale bylo i ve společném query, nebude patrně žádný problém.
query = '''query Parking (
        $from_lat: Float!,              # definování proměnných, které se vkládají ve "variables"
        $from_lon: Float!, 
        $to_lat: Float!, 
        $to_lon: Float!,
        $Date: String!,
        $Time: String!,
        $window: Long!,
        $NumInt: Int!
        ) 
    {
    plan(
        from: {lat: $from_lat, lon: $from_lon}
        to: {lat: $to_lat, lon: $to_lon}
        date: $Date
        time: $Time                    
        searchWindow: $window           # v jakém časovém okně od zadaného času, bude hledat trasy (v s)
        numItineraries: $NumInt         # počet iterací
        transportModes: [               # definice dopravních prostředků
            {mode: WALK},              
            {mode: BUS},               
            {mode: TRAM}, 
            {mode: RAIL}, 
            {mode: SUBWAY}, 
            {mode: TROLLEYBUS}, 
            {mode: FERRY}
        ]
        ) {
            itineraries {
            start                       # datetime, timezone - Prague
            end                         # datetime, timezone - Prague 
            duration                    # in seconds
            legs {
                mode
                route {
                shortName
                }
                distance
                duration
                from {
                name
                lat
                lon
                }
                to {
                name
                lat
                lon
                }
                legGeometry { 
                length
                points
                }
            }
            }
        }
        }
'''
row_number_1 = 10000
row_number_2 = 80000

Paha_Kancelare = pd.read_csv("DATA/API_zdroje/Praha-kancelare.csv").iloc[:row_number_1]


limit_time = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)
limit_time_walk = 20*60 # V Praze jsou nemovitosti i v docházkové vzdálenosti a někteří lidé si pojedou ty 2 zastávky, ale někteří radši půjdou pěšky. Ale pochybuji, že déle jak 20 min

date = Dates[2]
time = "7:00"
print(date)


#for date in Dates[0]:
routes = []  
errors = []
for _, row in Paha_Kancelare.iterrows():

    # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
    from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["data_gpsCoord_lat"], row["data_gpsCoord_lon"], row["KANCELAR_X"], row["KANCELAR_Y"])
    
    try:
        payload = {
            "query": query,
            "variables": {
                "from_lat": float(from_lat),
                "from_lon": float(from_lon),
                "to_lat": float(to_lat),
                "to_lon": float(to_lon),
                "Date": date,
                "Time": time,
                "window": 1800,
                "NumInt": 5
                },
            "operationName": "Parking"
            }
            

            # volání API
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        data = response.json()

        print(f'{_}, {row["id"]},{row["data_address"]} - {row['KANCELAR']} ')

        # ------------------------TRANSFORMACE-------------------------

        # pomocné proměnné
                    
        duration_fast = duration(data)
        leg_num_low = leg_num(data)
        duration_walk = float('inf') # nastaveno na nekonečno

        fast_route = {}
        leg_low_route = {}
        walk_route = {}

        for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
            all_coords = build_coordination_string(iteration)
            geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

            itinerary = build_itinerary(iteration)

            transport_lines, transport_vehicles = build_transport_list(iteration)
            # transports - konkrétní linky
            # transport_vehicles - typy dopravních prostředků
            transfers = transfer_num(iteration)
                        
            # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
            time_walk_start = start_walk_time(iteration)
            time_walk_end = end_walk_time(iteration)

            # ------------ FAST ----------
            # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
            if int(iteration['duration']) < duration_fast or iteration_index == 0:
                duration_fast = int(iteration['duration'])
                fast_route = build_route_dict('nejrychlejsi', row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, Prague=True)
                    
            # -------------- LOW LEG NUMBER --------------------
            # tady je to jedno, ale v případě obcí je třeba ošetřit cestu na vlak (ta může být delší a pak se to dá oseknout)
            if len(iteration['legs']) < leg_num_low and transfers > 0:
                # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze
                    
                if time_walk_start <= limit_time and time_walk_end <= limit_time: 
                    # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                    leg_num_low = len(iteration['legs'])
                    leg_low_route = build_route_dict ("nejmene prestupu", row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,Prague=True)
                
            # walk
            if len(iteration['legs']) == 1 and iteration["duration"] <= limit_time_walk:
                if int(iteration['duration']) < duration_walk:
                    duration_walk = int(iteration['duration'])
                    walk_route = build_route_dict('chuze', row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, Prague=True)

        if leg_low_route:
            if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                routes.append(fast_route)
            else:
                routes.append(fast_route)
                routes.append(leg_low_route)
        else:
            routes.append(fast_route)

        if walk_route:
            if fast_route["POCET_PRESTUPU"] != walk_route["POCET_PRESTUPU"]: 
                routes.append(walk_route)

                
    except Exception:
        error_dict = {
            "id": row["id"],
            "data_address": row["data_address"],
            "KANCELAR": row["KANCELAR"],
            "DATUM": date,
            "CAS": time,
            "from_lat": from_lat,
            "from_lon":from_lon,
            "to_lat": to_lat,
            "to_lon": to_lon,
            "data": data
        }
        errors.append(error_dict)

routes = pd.DataFrame(routes)
address = f"DATA_HD/trasy_PRAHA_{date}_{row_number_1}"
routes.to_csv(f'{address}.csv', sep=';', na_rep='', index= False)

if errors:
    with open (f'{address}_errors.json', "w", encoding="UTF-8") as output_file:
        json.dump(errors, output_file, ensure_ascii=False, indent=4)
    errors = pd.DataFrame(errors)
    errors.to_csv(f'{address}_errors.csv', sep=';', na_rep='', index= False)

numbers = [date, row_number_1]
address_2 = f"DATA_HD/trasy_PRAHA_kontrola"
with open(f'{address_2}.txt', "a", encoding="UTF-8") as output_file_2:
    print(numbers, file=output_file_2)

NameError: name 'pd' is not defined

In [18]:
data['data']['plan']['itineraries'][0]['legs'][0]['mode']

'WALK'

In [8]:
# GraphQL query
query = '''query Parking (
        $from_lat: Float!,              # definování proměnných, které se vkládají ve "variables"
        $from_lon: Float!, 
        $to_lat: Float!, 
        $to_lon: Float!,
        $Date: String!,
        $Time: String!,
        $window: Long!,
        $NumInt: Int!
        ) 
    {
    plan(
        from: {lat: $from_lat, lon: $from_lon}
        to: {lat: $to_lat, lon: $to_lon}
        date: $Date
        time: $Time                    
        searchWindow: $window           # v jakém časovém okně od zadaného času, bude hledat trasy (v s)
        numItineraries: $NumInt         # počet iterací
        transportModes: [               # definice dopravních prostředků
            {mode: WALK},              
            {mode: BUS},               
            {mode: TRAM}, 
            {mode: RAIL}, 
            {mode: SUBWAY}, 
            {mode: TROLLEYBUS}, 
            {mode: FERRY}
        ]
        ) {
            itineraries {
            start                       # datetime, timezone - Prague
            end                         # datetime, timezone - Prague 
            duration                    # in seconds
            legs {
                mode
                route {
                shortName
                }
                distance
                duration
                from {
                name
                lat
                lon
                }
                to {
                name
                lat
                lon
                }
                legGeometry { 
                length
                points
                }
            }
            }
        }
        }
'''
row_number_1 = 50000
row_number_2 = 60000

PR_Kancelare = pd.read_csv("DATA/API_zdroje/Praha-kancelare.csv").iloc[row_number_1:row_number_2]


limit_time = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)
limit_time_walk = 20*60 # V Praze jsou nemovitosti i v docházkové vzdálenosti a někteří lidé si pojedou ty 2 zastávky, ale někteří radši půjdou pěšky. Ale pochybuji, že déle jak 20 min

# date = Dates[0]
time = "7:00"


for date in Dates:
    routes = []  
    errors = []
    for _, row in PR_Kancelare.iterrows():

        # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
            from_lat, from_lon, to_lat, to_lon, spicka = direction_decide(time, row["data_gpsCoord_lat"], row["data_gpsCoord_lon"], row["KANCELAR_X"], row["KANCELAR_Y"])
            print(f'{row["id"]},{row["data_address"]} - {row['KANCELAR']} ')
            payload = {
                "query": query,
                "variables": {
                    "from_lat": float(from_lat),
                    "from_lon": float(from_lon),
                    "to_lat": float(to_lat),
                    "to_lon": float(to_lon),
                    "Date": date,
                    "Time": time,
                    "window": 1800,
                    "NumInt": 5
                    },
                "operationName": "Parking"
                }
            

            # volání API
            response = requests.post(url, headers=headers, json=payload, timeout=30)
            data = response.json()

            # ------------------------TRANSFORMACE-------------------------

            # pomocné proměnné
                    
            duration_fast = duration(data)
            leg_num_low = leg_num(data)
            duration_walk = float('inf')

            fast_route = {}
            leg_low_route = {}
            walk_route = {}

            for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
                all_coords = build_coordination_string(iteration)
                geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

                itinerary = build_itinerary(iteration)

                transport_lines, transport_vehicles = build_transport_list(iteration)
                # transports - konkrétní linky
                # transport_vehicles - typy dopravních prostředků
                transfers = transfer_num(iteration)
                        
                # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
                time_walk_start = start_walk_time(iteration)
                time_walk_end = end_walk_time(iteration)

                # ------------ FAST ----------
                # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
                if int(iteration['duration']) < duration_fast or iteration_index == 0:
                    duration_fast = int(iteration['duration'])
                    fast_route = build_route_dict('nejrychlejsi', row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, Prague=True)
                # -------------- LOW LEG NUMBER --------------------
                # tady je to jedno, ale v případě obcí je třeba ošetřit cestu na vlak (ta může být delší a pak se to dá oseknout)
                if len(iteration['legs']) < leg_num_low and transfers > 0:
                    # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze
                    if time_walk_start <= limit_time and time_walk_end <= limit_time: 
                        # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                        leg_num_low = len(iteration['legs'])
                        leg_low_route = build_route_dict ("nejmene prestupu", row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,Prague=True)
                
                # walk
                if len(iteration['legs']) == 1 and iteration["duration"] <= limit_time_walk:
                    if not walk_route:
                        duration_walk = int(iteration['duration'])
                        walk_route = build_route_dict('chuze', row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, Prague=True)
                    elif int(iteration['duration']) < duration_walk:
                        duration_walk = int(iteration['duration'])
                        walk_route = build_route_dict('chuze', row, "KANCELAR", "id", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, Prague=True)

            if leg_low_route:
                if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                    routes.append(fast_route)
                else:
                    routes.append(fast_route)
                    routes.append(leg_low_route)
            else:
                routes.append(fast_route)

            if walk_route:
                if fast_route["POCET_PRESTUPU"] != walk_route["POCET_PRESTUPU"]: 
                    routes.append(walk_route)

668e6035cf63967a8ac29918,Nad lesním divadlem, Praha 4 - Braník - Smíchov 
6807525ed6e2288a7192b0cd,Bínova, Praha 8 - Střížkov - Brumlovka 
6807525ed6e2288a7192b0cd,Bínova, Praha 8 - Střížkov - Karlín 
6807525ed6e2288a7192b0cd,Bínova, Praha 8 - Střížkov - Smíchov 
67b9e87f92ccc45937d098fb,Anny Drabíkové, Praha - Háje - Brumlovka 
67b9e87f92ccc45937d098fb,Anny Drabíkové, Praha - Háje - Karlín 
67b9e87f92ccc45937d098fb,Anny Drabíkové, Praha - Háje - Smíchov 
6863c446511ec6e4d7c94cd2,Rohanské nábřeží, Praha 8 - Karlín - Brumlovka 
6863c446511ec6e4d7c94cd2,Rohanské nábřeží, Praha 8 - Karlín - Karlín 
6863c446511ec6e4d7c94cd2,Rohanské nábřeží, Praha 8 - Karlín - Smíchov 
65aa40c4c97da6715486f6ad,Na vysoké I, Praha 5 - Brumlovka 
65aa40c4c97da6715486f6ad,Na vysoké I, Praha 5 - Karlín 
65aa40c4c97da6715486f6ad,Na vysoké I, Praha 5 - Smíchov 
6798fea766a63bef62f97351,Šumberova, Praha 6 - Veleslavín - Brumlovka 
6798fea766a63bef62f97351,Šumberova, Praha 6 - Veleslavín - Karlín 
6798fea766a63bef6

IndexError: list index out of range

In [37]:
routes = pd.DataFrame(routes)
routes

,KATEGORIE_ITERACE,data_address,ZASTAVKA_JMENO,ZASTAVKA_lat,ZASTAVKA_lon,KANCELAR,id,DATUM A CAS,SPICKA,DOBA,CAS_ZACATEK,CAS_KONEC,DELKA,POCET_PRESTUPU,DOPRAVNI_PROSTREDKY,TRASA,CISLO_ITERACE,KROKY,LINESTRINGY_ETAPY
0,nejrychlejsi,Praha,Hlavní nádraží,50.083515,14.434089,Brumlovka,xhh5imi48EKvmmhtB,2025-11-14T7:00,ranni,19,07:07:18,07:26:43,6.2,1,"BUS, SUBWAY","[(14.43411, 50.08355), (14.43381, 50.08332), (...",2,"WALK, 2 min, 97.93 m (Origin - Hlavní nádraží)...","WALK+[(14.43411, 50.08355), (14.43381, 50.0833..."
1,nejrychlejsi,Praha,Hlavní nádraží,50.083495,14.434156,Karlín,xhh5imi48EKvmmhtB,2025-11-14T7:00,ranni,16,07:03:37,07:19:40,2.4,1,"SUBWAY, TRAM","[(14.43411, 50.08355), (14.43381, 50.08332), (...",2,"WALK, 2 min, 112.9 m (Origin - Hlavní nádraží)...","WALK+[(14.43411, 50.08355), (14.43381, 50.0833..."
2,nejrychlejsi,Praha,Hlavní nádraží,50.083495,14.434156,Smíchov,xhh5imi48EKvmmhtB,2025-11-14T7:00,ranni,23,07:03:37,07:26:57,5.3,1,SUBWAY,"[(14.43411, 50.08355), (14.43381, 50.08332), (...",3,"WALK, 2 min, 112.9 m (Origin - Hlavní nádraží)...","WALK+[(14.43411, 50.08355), (14.43381, 50.0833..."
3,nejrychlejsi,"Holečkova, Praha 5 - Smíchov",Bertramka,50.072231,14.393291,Brumlovka,6898e28419c7639be733ba68,2025-11-14T7:00,ranni,40,07:07:25,07:47:43,8.6,2,"BUS, SUBWAY, TRAM","[(14.39456, 50.07396), (14.39489, 50.07408), (...",3,"WALK, 7 min, 433.72 m (Origin - Bertramka) + T...","WALK+[(14.39456, 50.07396), (14.39489, 50.0740..."
4,nejrychlejsi,"Holečkova, Praha 5 - Smíchov",Anděl,50.070001,14.403957,Karlín,6898e28419c7639be733ba68,2025-11-14T7:00,ranni,31,07:00:46,07:31:41,6.3,0,SUBWAY,"[(14.39456, 50.07396), (14.39489, 50.07408), (...",2,"WALK, 15 min, 1045.18 m (Origin - Anděl) + SUB...","WALK+[(14.39456, 50.07396), (14.39489, 50.0740..."
5,nejrychlejsi,"Holečkova, Praha 5 - Smíchov",NaN,NaN,NaN,Smíchov,6898e28419c7639be733ba68,2025-11-14T7:00,ranni,11,07:00:00,07:10:35,0.7,-1,,"[(14.39456, 50.07396), (14.39489, 50.07408), (...",1,"WALK, 11 min, 722.93 m (Origin - Destination)","WALK+[(14.39456, 50.07396), (14.39489, 50.0740..."
6,nejrychlejsi,"Za Zelenou liškou, Praha 4 - Krč",Budějovická,50.044437,14.447827,Brumlovka,65afa308f71b0bb3f34983e6,2025-11-14T7:00,ranni,12,07:05:51,07:17:43,1.4,0,BUS,"[(14.44323, 50.04626), (14.44324, 50.04625), (...",4,"WALK, 7 min, 529.65 m (Origin - Budějovická) +...","WALK+[(14.44323, 50.04626), (14.44324, 50.0462..."
7,nejrychlejsi,"Za Zelenou liškou, Praha 4 - Krč",Budějovická,50.044459,14.448829,Karlín,65afa308f71b0bb3f34983e6,2025-11-14T7:00,ranni,31,07:00:39,07:31:41,8.0,1,SUBWAY,"[(14.44323, 50.04626), (14.44324, 50.04625), (...",2,"WALK, 7 min, 524.49 m (Origin - Budějovická) +...","WALK+[(14.44323, 50.04626), (14.44324, 50.0462..."
8,nejrychlejsi,"Za Zelenou liškou, Praha 4 - Krč",Budějovická,50.044459,14.448829,Smíchov,65afa308f71b0bb3f34983e6,2025-11-14T7:00,ranni,33,07:06:29,07:39:32,6.9,1,"SUBWAY, TRAM","[(14.44323, 50.04626), (14.44324, 50.04625), (...",4,"WALK, 7 min, 524.49 m (Origin - Budějovická) +...","WALK+[(14.44323, 50.04626), (14.44324, 50.0462..."
9,nejrychlejsi,"Jablonecká, Praha 9 - Střížkov",Střížkov,50.126178,14.489525,Brumlovka,6639b20437fcd7ec891f540b,2025-11-14T7:00,ranni,38,07:03:10,07:40:43,15.1,1,"BUS, SUBWAY","[(14.49482, 50.12433), (14.4948, 50.12434), (1...",2,"WALK, 6 min, 440.8 m (Origin - Střížkov) + SUB...","WALK+[(14.49482, 50.12433), (14.4948, 50.12434..."


# Erorry

In [38]:
url = "http://localhost:8080/otp/gtfs/v1"
headers = {"Content-Type": "application/json"}

# GraphQL query
query = '''query Parking (
        $from_lat: Float!,              # definování proměnných, které se vkládají ve "variables"
        $from_lon: Float!, 
        $to_lat: Float!, 
        $to_lon: Float!,
        $Date: String!,
        $Time: String!,
        $window: Long!,
        $NumInt: Int!
        ) 
    {
    plan(
        from: {lat: $from_lat, lon: $from_lon}
        to: {lat: $to_lat, lon: $to_lon}
        date: $Date
        time: $Time                    
        searchWindow: $window           # v jakém časovém okně od zadaného času, bude hledat trasy (v s)
        numItineraries: $NumInt         # počet iterací
        # maxTripDurationSeconds: 14400
        # maxNumberOfTransfers: 40
        maxWalkDistance: 500000
        transportModes: [               # definice dopravních prostředků
            {mode: WALK},              
            {mode: BUS},               
            {mode: TRAM}, 
            {mode: RAIL}, 
            {mode: SUBWAY}, 
            {mode: TROLLEYBUS}, 
            {mode: FERRY}
        ]
        ) {
            itineraries {
            start                       # datetime, timezone - Prague
            end                         # datetime, timezone - Prague 
            duration                    # in seconds
            legs {
                mode
                route {
                shortName
                }
                distance
                duration
                from {
                name
                }
                to {
                name
                }
                legGeometry { 
                length
                points
                }
            }
            }
        }
        }
'''

obce = pd.read_csv("DATA_HD/trasy_obce_2025-11_10-14_errors.csv", sep=';')

routes = []
rail_routes = []

errors = []

limit_time_Prague = 10*60 # 10 min, nastavení limitu pro chůzi z výchozích/do koncových bodů, pro Prahu je to 10 (občas to hází, že  je rychlejší jít 40 min než čekat na dopravní prostředky)
limit_time_not_Prague = 20*60 # 20 min, omezení mimo Prahu

for _, row in obce.iterrows():
    date = row['DATUM']
    time = row["CAS"]

    # Rozhodnutí, jestli se jede do kanceláře nebo z kanceláře
    from_lat, from_lon, to_lat, to_lon = row['from_lat'], row['from_lon'], row['to_lat'], row['to_lon']
    if datetime.strptime(time, "%H:%M").hour < 12:
        spicka = "ranni"
    else:
        spicka = "odpoledni"
    
    try:    
        payload = {
            "query": query,
            "variables": {
                "from_lat": float(from_lat),
                "from_lon": float(from_lon),
                "to_lat": float(to_lat),
                "to_lon": float(to_lon),
                "Date": date,
                "Time": time,
                "window": 86400,
                "NumInt": 5
                },
            "operationName": "Parking"
            }
                        
        # volání API
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()

        # ------------------------TRANSFORMACE-------------------------
        print(f'{row["OBEC_NAZEV"]} - {row["KANCELAR"]}')

        # pomocné proměnné 

        duration_fast = duration(data)
        rail_route_fast = None 
        leg_num_low = leg_num(data)

        fast_route = {}
        leg_low_route = {}
        rail_route = {}

        for iteration_index, iteration in enumerate(data['data']['plan']['itineraries']):
            all_coords = build_coordination_string(iteration)
            geometry_legs = build_geometry_legs_string(iteration) # geometrie jednotlivých kroků + dopravní prostředek

            itinerary = build_itinerary(iteration)

            transport_lines, transport_vehicles = build_transport_list(iteration)
            transfers = transfer_num(iteration)
                            
            # na začátku a konci je vždy chůze, pro zjistění, jestli nejsou moc dlouhé
            time_walk_start = start_walk_time(iteration)
            time_walk_end = end_walk_time(iteration)

            # ------------ FAST ------------------------------
            # rozhodovací bloky - rozdělení na nejrychlejší a s nejméně přestupy
            if int(iteration['duration']) < duration_fast or iteration_index == 0:
                duration_fast = int(iteration['duration'])
                # fast_transports = transport_lines
                fast_route = build_route_dict('nejrychlejsi', row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=True)
                    
            # -------------- LOW LEG NUMBER --------------------
            if len(iteration['legs']) < leg_num_low and transfers > 0:
                # počet kroků, kde je více než 2 vynechá iterace, kde je jen chůze

                # protože rozlišuji chůzi v Praze a v obci, je třeba nejdřív rozdělit směr
                if spicka == "ranni": # trasa obce -> kancelare
                    # takže time_walk_start - má limit_time_not_Prague
                    if time_walk_start <= limit_time_not_Prague and time_walk_end <= limit_time_Prague: 
                    # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                        leg_num_low = len(iteration['legs'])
                        leg_low_route = build_route_dict("nejmene prestupu", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs, municipalities=True)
                else:
                    # tady je to obrácene, tedy time_walk_start má limit_time_Prague
                    if time_walk_start <= limit_time_Prague and time_walk_end <= limit_time_not_Prague: 
                        # omezení na to, aby chůze na začátku nebo na konci nebyla delší než je stanovený limit, plus tady je to důležitý jen aby se pojistilo, že nejméně přestupů je skutečně u nejrychlejší trasy. Větší smysl to bude u obcí, kde je větší šance, že rychlý čas je kvůli velkému počtu přestupů. Ale kdyby tady celý blok nebyl, vůbec se nic nestane
                        leg_num_low = len(iteration['legs'])
                        leg_low_route = build_route_dict("nejmene prestupu", row, "OBEC_NAZEV", "KANCELAR", date, time, spicka, iteration, iteration_index, all_coords, transfers, transport_vehicles, itinerary, geometry_legs,municipalities=True)
                        

        if leg_low_route:
            if fast_route["POCET_PRESTUPU"] == leg_low_route["POCET_PRESTUPU"]: 
                routes.append(fast_route)
            else:
                routes.append(fast_route)
                routes.append(leg_low_route)
        else:
                routes.append(fast_route)
        
    except Exception:
        error_dict = {
            "OBEC_NAZEV": row["OBEC_NAZEV"],
            "OKRES_KOD": row["OKRES_KOD"],
            "KANCELAR": row["KANCELAR"],
            "DATUM": date,
            "CAS": time,
            "from_lat": from_lat,
            "from_lon":from_lon,
            "to_lat": to_lat,
            "to_lon": to_lon,
            "data": data
        }
        errors.append(error_dict)

routes = pd.DataFrame(routes)
adress = f"DATA_HD/trasy_obce_error_oprava"
routes.to_csv(f"{adress}.csv", sep=';', na_rep='', index= False)

if errors:
    with open (f'{adress}_errors.json', "w", encoding="UTF-8") as output_file:
        json.dump(errors, output_file, ensure_ascii=False, indent=4)
    errors = pd.DataFrame(errors)
    errors.to_csv(f'{adress}_errors.csv', sep=';', na_rep='', index= False)

Vysoký Chlumec - Karlín
Vysoký Chlumec - Smíchov
Drobovice - Brumlovka
Drobovice - Karlín
Drobovice - Smíchov
Děkov - Brumlovka
Děkov - Karlín
Děkov - Smíchov
Košík - Brumlovka
Košík - Karlín
Košík - Smíchov
Velká Chmelištná - Brumlovka
Velká Chmelištná - Karlín
Velká Chmelištná - Smíchov
Zduchovice - Brumlovka
Zduchovice - Karlín
Zduchovice - Smíchov
Bílé Podolí - Brumlovka
Bílé Podolí - Karlín
Bílé Podolí - Smíchov
Kounice - Brumlovka
Kácov - Brumlovka
Kácov - Karlín
Kácov - Smíchov
Louňovice pod Blaníkem - Brumlovka
Louňovice pod Blaníkem - Karlín
Louňovice pod Blaníkem - Smíchov
Maršovice - Brumlovka
Maršovice - Karlín
Maršovice - Smíchov
Načeradec - Brumlovka
Načeradec - Karlín
Načeradec - Smíchov
Pavlíkov - Brumlovka
Pavlíkov - Karlín
Pavlíkov - Smíchov
Český Šternberk - Brumlovka
Český Šternberk - Karlín
Český Šternberk - Smíchov
Pečice - Brumlovka
Pečice - Karlín
Pečice - Smíchov
Bernartice - Brumlovka
Bernartice - Karlín
Bernartice - Smíchov
Blažejovice - Brumlovka
Blažejovice

In [35]:
routes

,KATEGORIE_ITERACE,OKRES_KOD,OBEC_NAZEV,KANCELAR,DATUM A CAS,SPICKA,DOBA,DELKA,POCET_PRESTUPU,DOPRAVNI_PROSTREDKY,TRASA,CISLO_ITERACE,CAS_ZACATEK,CAS_KONEC,KROKY,LINESTRINGY_ETAPY
0,nejrychlejsi,CZ0205,Drobovice,Brumlovka,2025-11-10T06:30,ranni,164,93.5,1,RAIL,"[(15.41436, 49.88993), (15.41439, 49.88993), (...",2,07:57:39,10:41:32,"WALK, 41 min, 3274.37 m (Origin - Vrdy-Koudelo...","WALK+[(15.41436, 49.88993), (15.41439, 49.8899..."
1,nejrychlejsi,CZ0205,Drobovice,Karlín,2025-11-10T06:30,ranni,141,89.9,1,RAIL,"[(15.41436, 49.88993), (15.41439, 49.88993), (...",2,07:57:39,10:18:45,"WALK, 41 min, 3274.37 m (Origin - Vrdy-Koudelo...","WALK+[(15.41436, 49.88993), (15.41439, 49.8899..."
2,nejrychlejsi,CZ0205,Drobovice,Smíchov,2025-11-10T06:30,ranni,155,94.4,2,RAIL,"[(15.41436, 49.88993), (15.41439, 49.88993), (...",2,07:57:39,10:33:01,"WALK, 41 min, 3274.37 m (Origin - Vrdy-Koudelo...","WALK+[(15.41436, 49.88993), (15.41439, 49.8899..."
